In [155]:
from sklearn.datasets import fetch_openml
import pickle
import os

In [156]:
def load_mnist():
    pickle_path = '../data/mnist/data.pkl'
    if os.path.exists(pickle_path):
        with open(pickle_path, 'rb') as f:
            return pickle.load(f)
    mnist = fetch_openml(name='mnist_784', version=1, parser="auto")
    with open(pickle_path, 'wb') as f:
        pickle.dump(mnist, f)
    return mnist

mnist = load_mnist()

In [157]:
from random import shuffle
from jax import numpy as jnp
import pandas as pd

all_features = mnist['data']
all_targets = mnist['target']

bool_vec = [i < 60_000 for i in range(len(all_targets))]
shuffle(bool_vec)
split_df = lambda df : (
    df[pd.Series(bool_vec).values],
    df[pd.Series([not b for b in bool_vec]).values]
)

train_features, test_features = split_df(all_features)
train_targets, test_targets = split_df(all_targets)

format_jnp = lambda *dfs : tuple([jnp.asarray(df.to_numpy(), dtype='float32') for df in dfs])

ftr_train, ftr_test, tgt_train, tgt_test = format_jnp(
    train_features,
    test_features,
    pd.get_dummies(train_targets),
    pd.get_dummies(test_targets)
)

normalize = lambda ftr_df : ftr_df / jnp.linalg.norm(ftr_df, axis=1, keepdims=True)
norm_ftr_train, norm_ftr_test = normalize(ftr_train), normalize(ftr_test)

"""
Quick sanity check to make sure that features have been properly normalized
"""

is_normalized = all(jnp.isclose(
    jnp.linalg.norm(norm_ftr_train, axis=1),
    jnp.full(norm_ftr_train.shape[0], 1.0)
))

assert(is_normalized)

In [158]:
from random import randint
from jax import random as jrand

class NormalSampler:
    def __init__(self):
        self.key = jrand.PRNGKey(randint(0, 10**6))
        
    def sample(self, *shape):
        self.key, subkey = jrand.split(self.key)
        return jrand.normal(subkey, shape=tuple(shape))
    
normal_sampler = NormalSampler()

In [222]:
from jax import grad, nn, jit

def tanh_layer(x, w, b):
    # x has dims (#In,) and w has dims (#Out, #In) and b has dims (#Out,)
    return jnp.tanh(
        jnp.dot(w, x) + b
    )

def relu_softmax_layer(x, w, b):
    # x has dims (#In,) and w has dims (#Out, #In) and b has dims (#Out,)
    return nn.softmax(nn.relu(
        jnp.dot(w, x) + b
    ))

def neural_net(x, params):
    w1, b1, w2, b2, w3, b3 = params
    t0 = relu_softmax_layer(x, w1, b1)
    t1 = relu_softmax_layer(t0, w2, b2)
    t2 = relu_softmax_layer(t1, w3, b3)
    # return nn.softmax(t2)
    return t2

# MSE
# def loss_fn(params, x, y):
#     result = neural_net(x, params)
#     return jnp.linalg.norm(y - result)

# Cross entropy
def loss_fn(params, x, y):
    out = neural_net(x, params)
    cross_entropy_vec = -y * jnp.log(out) - (1 - y)*jnp.log(1 - out)
    ce = jnp.sum(cross_entropy_vec)
    return ce

def show_cross_entropy(out, y):
    cross_entropy_vec = -y * jnp.log(out) - (1 - y)*jnp.log(1 - out)
    ce = jnp.sum(cross_entropy_vec)
    print(f'out = {out}')
    print(f'y = {y}')
    print(f'ce_vec = {cross_entropy_vec}')
    print(f'ce = {ce}')
    return ce

# GOTCHA
# by default, only the first paramter of the input function
# will be differentiated against
loss_gradient = jit(grad(loss_fn))
jit_nn = jit(neural_net)

In [223]:
num_obs, num_ftrs = ftr_train.shape

def sgd_sequence(params, batch_size, learning_rate=0.01):
    for _ in range(batch_size):
        idx = randint(0, num_obs - 1)
        x, y = ftr_train[idx], tgt_train[idx]
        grads = loss_gradient(params, x, y)
        params = [
            param_vec - (learning_rate * grad)
            for param_vec, grad in zip(params, grads)
        ]
    return params

def sgd_minibatch(params, batch_size, learning_rate=0.01):
    new_params = [jnp.copy(p) for p in params]
    for _ in range(batch_size):
        idx = randint(0, num_obs - 1)
        x, y = ftr_train[idx], tgt_train[idx]
        grads = loss_gradient(params, x, y)
        new_params = [
            param_vec - ((learning_rate / batch_size) * grad)
            for param_vec, grad in zip(new_params, grads)
        ]
    return new_params

def avg_training_loss(params):
    loss = 0
    n = 100
    for _ in range(n):
        idx = randint(0, num_obs - 1)
        x, y = ftr_train[idx], tgt_train[idx]
        loss += loss_fn(params, x, y)
    return loss / n

def _max_idx(vec):
    max_x = vec[0]
    max_i = 0
    for i, x in enumerate(vec):
        if vec[i] > max_x:
            max_i = i
    return max_i
        
def show_prediction_accuracy(params):
    correct = 0
    n = 1000
    for idx in range(n):
        x, y = ftr_test[idx], tgt_test[idx]
        pred_vec = jit_nn(x, params)
        if _max_idx(pred_vec) == _max_idx(y):
            correct += 1
    print(f'{correct}/{n}')
    x, y = ftr_test[0], tgt_test[0]
    pred_vec = jit_nn(x, params)
    show_cross_entropy(pred_vec, y)

In [224]:
params = [
    normal_sampler.sample(784, num_ftrs), # w1
    normal_sampler.sample(784), # b1
    normal_sampler.sample(50, 784), # w2
    normal_sampler.sample(50), #b2
    normal_sampler.sample(10, 50), # w3
    normal_sampler.sample(10), #b3
]

for epoch_num in range(10000):
    params = sgd_minibatch(params, 1_000, 0.2)
    loss = avg_training_loss(params)
    print(f'current average loss: {loss}')
    #show_prediction_accuracy(params)

current average loss: 3.5234062671661377
current average loss: 3.52799654006958
current average loss: 3.586683750152588
current average loss: 3.6593925952911377
current average loss: 3.4805567264556885
current average loss: 3.432021141052246
current average loss: 3.3682944774627686
current average loss: 3.4592959880828857
current average loss: 3.5953280925750732
current average loss: 3.4178261756896973
current average loss: 3.5407423973083496
current average loss: 3.566450834274292
current average loss: 3.398486614227295
current average loss: 3.4824166297912598
current average loss: 3.3253660202026367
current average loss: 3.471677780151367
current average loss: 3.4549007415771484
current average loss: 3.337700605392456
current average loss: 3.379695177078247
current average loss: 3.493037462234497
current average loss: 3.4345924854278564
current average loss: 3.336812973022461
current average loss: 3.375241994857788
current average loss: 3.434279441833496
current average loss: 3.36144

current average loss: 3.250150680541992
current average loss: 3.2599802017211914
current average loss: 3.2369353771209717
current average loss: 3.2445995807647705
current average loss: 3.244103193283081
current average loss: 3.2216451168060303
current average loss: 3.2317209243774414
current average loss: 3.2370691299438477
current average loss: 3.2349133491516113
current average loss: 3.237431049346924
current average loss: 3.232419729232788
current average loss: 3.256964921951294
current average loss: 3.252925157546997
current average loss: 3.2451021671295166
current average loss: 3.2223596572875977
current average loss: 3.2458906173706055
current average loss: 3.2648134231567383
current average loss: 3.2265002727508545
current average loss: 3.239595651626587
current average loss: 3.267719030380249
current average loss: 3.222245693206787
current average loss: 3.2548580169677734
current average loss: 3.2515978813171387
current average loss: 3.2387757301330566
current average loss: 3.2

current average loss: 3.203615665435791
current average loss: 3.218012809753418
current average loss: 3.2198963165283203
current average loss: 3.251607894897461
current average loss: 3.2161219120025635
current average loss: 3.234081745147705
current average loss: 3.2169911861419678
current average loss: 3.2435617446899414
current average loss: 3.250396966934204
current average loss: 3.218717098236084
current average loss: 3.205861806869507
current average loss: 3.215425968170166
current average loss: 3.2164342403411865
current average loss: 3.222754716873169
current average loss: 3.2103629112243652
current average loss: 3.209228515625
current average loss: 3.2299206256866455
current average loss: 3.240769147872925
current average loss: 3.1978485584259033
current average loss: 3.2109768390655518
current average loss: 3.211763620376587
current average loss: 3.2205638885498047
current average loss: 3.1914756298065186
current average loss: 3.2301719188690186
current average loss: 3.2368426

current average loss: 3.1794283390045166
current average loss: 3.2015576362609863
current average loss: 3.2129080295562744
current average loss: 3.1878409385681152
current average loss: 3.189765214920044
current average loss: 3.1792855262756348
current average loss: 3.2036938667297363
current average loss: 3.1823835372924805
current average loss: 3.235447883605957
current average loss: 3.1525511741638184
current average loss: 3.2270727157592773
current average loss: 3.205392837524414
current average loss: 3.1878368854522705
current average loss: 3.239635944366455
current average loss: 3.1854445934295654
current average loss: 3.1780402660369873
current average loss: 3.1770858764648438
current average loss: 3.2015795707702637
current average loss: 3.231224775314331
current average loss: 3.1509523391723633
current average loss: 3.1831271648406982
current average loss: 3.1904990673065186
current average loss: 3.224297285079956
current average loss: 3.1700263023376465
current average loss: 

current average loss: 3.1538519859313965
current average loss: 3.119495153427124
current average loss: 3.0872159004211426
current average loss: 3.0950984954833984
current average loss: 3.087486982345581
current average loss: 3.152348041534424
current average loss: 3.1334545612335205
current average loss: 3.110990047454834
current average loss: 3.13631534576416
current average loss: 3.1662182807922363
current average loss: 3.1491811275482178
current average loss: 3.1458988189697266
current average loss: 3.090684175491333
current average loss: 3.093374013900757
current average loss: 3.1189301013946533
current average loss: 3.028660297393799
current average loss: 3.068902015686035
current average loss: 3.0144143104553223
current average loss: 3.056849956512451
current average loss: 3.1643974781036377
current average loss: 3.0969197750091553
current average loss: 3.1165735721588135
current average loss: 3.096712112426758
current average loss: 3.116752862930298
current average loss: 3.09739

...

current average loss: 1.4948468208312988
current average loss: 1.6668678522109985
current average loss: 2.112032890319824
current average loss: 2.0016372203826904
current average loss: 1.3478312492370605
current average loss: 1.7065998315811157
current average loss: 1.597445011138916
current average loss: 1.7704359292984009
current average loss: 1.373231053352356
current average loss: 1.3628484010696411
current average loss: 1.2259562015533447
current average loss: 1.5195802450180054
current average loss: 1.9697532653808594
current average loss: 1.41031813621521
current average loss: 1.753832221031189
current average loss: 2.0405571460723877
current average loss: 1.422714114189148
current average loss: 1.7760441303253174
current average loss: 1.8401492834091187
current average loss: 1.4475756883621216
current average loss: 1.7355198860168457
current average loss: 1.9232003688812256
current average loss: 1.5727685689926147
current average loss: 1.7007347345352173
current average loss: 1

current average loss: 2.0720016956329346
current average loss: 1.5105613470077515
current average loss: 1.3076047897338867
current average loss: 1.6567779779434204
current average loss: 1.5136107206344604
current average loss: 1.610039472579956
current average loss: 1.5252070426940918
current average loss: 1.5742754936218262
current average loss: 1.7346444129943848
current average loss: 1.4065192937850952
current average loss: 1.362109661102295
current average loss: 1.7638360261917114
current average loss: 1.711747407913208
current average loss: 1.2558507919311523
current average loss: 1.6019498109817505
current average loss: 1.4625325202941895
current average loss: 1.2317283153533936
current average loss: 1.5907875299453735
current average loss: 1.7983076572418213
current average loss: 1.577681303024292
current average loss: 1.7840014696121216
current average loss: 1.6307247877120972
current average loss: 1.732977032661438
current average loss: 1.5195311307907104
current average loss:

current average loss: 1.6112141609191895
current average loss: 2.0012013912200928
current average loss: 1.7419335842132568
current average loss: 2.2300074100494385
current average loss: 1.8181079626083374
current average loss: 1.8962103128433228
current average loss: 1.6084877252578735
current average loss: 1.557542324066162
current average loss: 1.5791500806808472
current average loss: 1.824463963508606
current average loss: 2.140589952468872
current average loss: 1.8381158113479614
current average loss: 1.8538175821304321
current average loss: 1.2535494565963745
current average loss: 1.694257378578186
current average loss: 1.6274586915969849
current average loss: 1.6962652206420898
current average loss: 1.5341415405273438
current average loss: 1.5002317428588867
current average loss: 1.8823013305664062
current average loss: 1.7414963245391846
current average loss: 1.6278445720672607
current average loss: 1.8569499254226685
current average loss: 1.4555153846740723
current average loss

current average loss: 1.6443949937820435
current average loss: 1.5869529247283936
current average loss: 1.7547513246536255
current average loss: 1.671114444732666
current average loss: 1.8210959434509277
current average loss: 1.5307730436325073
current average loss: 1.6120457649230957
current average loss: 1.7627941370010376
current average loss: 1.9210280179977417
current average loss: 1.8232098817825317
current average loss: 1.7180311679840088
current average loss: 1.432504415512085
current average loss: 1.8029659986495972
current average loss: 1.4808249473571777
current average loss: 1.7159438133239746
current average loss: 1.794918179512024
current average loss: 1.6532613039016724
current average loss: 1.293753981590271
current average loss: 1.7221249341964722
current average loss: 1.5135923624038696
current average loss: 1.3254714012145996
current average loss: 1.549487590789795
current average loss: 1.5836234092712402
current average loss: 1.7862794399261475
current average loss:

current average loss: 1.8008755445480347
current average loss: 1.3180958032608032
current average loss: 1.526045799255371
current average loss: 1.502625584602356
current average loss: 1.5334566831588745
current average loss: 1.4943729639053345
current average loss: 1.7171189785003662
current average loss: 1.7221133708953857
current average loss: 1.6643688678741455
current average loss: 1.943538784980774
current average loss: 1.2029205560684204
current average loss: 1.5112844705581665
current average loss: 1.5842379331588745
current average loss: 1.684096336364746
current average loss: 1.466538667678833
current average loss: 1.5048350095748901
current average loss: 1.5841962099075317
current average loss: 1.7553356885910034
current average loss: 1.8458044528961182
current average loss: 2.0912036895751953
current average loss: 1.5655778646469116
current average loss: 1.8335763216018677
current average loss: 1.5676567554473877
current average loss: 1.606687307357788
current average loss: 

In [225]:
show_prediction_accuracy(params)

361/1000
out = [0.9544837  0.00505736 0.00505736 0.00505736 0.00505736 0.00505736
 0.00505736 0.00505736 0.00505736 0.00505736]
y = [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ce_vec = [0.04658473 0.00507017 0.00507017 0.00507017 0.00507017 0.00507017
 0.00507017 0.00507017 0.00507017 0.00507017]
ce = 0.09221623092889786


In [227]:
for epoch_num in range(10000):
    params = sgd_minibatch(params, 1_000, 0.2)
    loss = avg_training_loss(params)
    print(f'current average loss: {loss}')

current average loss: 1.7391780614852905
current average loss: 1.991186499595642
current average loss: 1.7096425294876099
current average loss: 1.564477562904358
current average loss: 1.5122636556625366
current average loss: 1.5023400783538818
current average loss: 1.6202393770217896
current average loss: 1.672518253326416
current average loss: 1.393739938735962
current average loss: 1.2947299480438232
current average loss: 1.7932757139205933
current average loss: 1.175085425376892
current average loss: 1.7098830938339233
current average loss: 1.1554492712020874
current average loss: 1.3912577629089355
current average loss: 1.2774913311004639
current average loss: 1.519317626953125
current average loss: 1.6433566808700562
current average loss: 1.8087536096572876
current average loss: 1.6793196201324463
current average loss: 1.5354549884796143
current average loss: 1.3937219381332397
current average loss: 1.5327857732772827
current average loss: 1.9570561647415161
current average loss: 

current average loss: 1.6326587200164795
current average loss: 1.3592617511749268
current average loss: 1.5722706317901611
current average loss: 1.7956372499465942
current average loss: 1.560060977935791
current average loss: 1.598343849182129
current average loss: 1.5624803304672241
current average loss: 1.651181697845459
current average loss: 1.4635963439941406
current average loss: 1.7695879936218262
current average loss: 1.3092705011367798
current average loss: 1.6623578071594238
current average loss: 1.4959819316864014
current average loss: 1.7335247993469238
current average loss: 1.5422215461730957
current average loss: 1.671675682067871
current average loss: 1.7440462112426758
current average loss: 1.2731574773788452
current average loss: 1.8372411727905273
current average loss: 1.42071533203125
current average loss: 1.5907591581344604
current average loss: 1.636791706085205
current average loss: 1.7846542596817017
current average loss: 1.672937035560608
current average loss: 1.

current average loss: 1.9294487237930298
current average loss: 1.2713857889175415
current average loss: 1.406662940979004
current average loss: 1.5464924573898315
current average loss: 1.6624770164489746
current average loss: 1.5726065635681152
current average loss: 1.5232958793640137
current average loss: 1.6358006000518799
current average loss: 2.005007743835449
current average loss: 1.6465002298355103
current average loss: 1.516535997390747
current average loss: 1.4643632173538208
current average loss: 1.650631308555603
current average loss: 1.3759469985961914
current average loss: 1.8142400979995728
current average loss: 1.7049307823181152
current average loss: 1.3953899145126343
current average loss: 1.7550103664398193
current average loss: 1.446162462234497
current average loss: 1.8231205940246582
current average loss: 1.3999592065811157
current average loss: 1.8858237266540527
current average loss: 1.6005057096481323
current average loss: 1.6267508268356323
current average loss:

current average loss: 1.5974295139312744
current average loss: 1.8329074382781982
current average loss: 1.8884114027023315
current average loss: 1.9621078968048096
current average loss: 1.4913222789764404
current average loss: 1.5319583415985107
current average loss: 1.5923675298690796
current average loss: 1.5603162050247192
current average loss: 1.9718011617660522
current average loss: 1.4115177392959595
current average loss: 1.5747134685516357
current average loss: 1.691917061805725
current average loss: 1.730047583580017
current average loss: 1.3876519203186035
current average loss: 1.8515738248825073
current average loss: 1.4564605951309204
current average loss: 1.3771705627441406
current average loss: 1.5362211465835571
current average loss: 1.6914857625961304
current average loss: 1.7568316459655762
current average loss: 1.5520023107528687
current average loss: 1.358955979347229
current average loss: 1.5669993162155151
current average loss: 1.4542412757873535
current average los

current average loss: 1.552804708480835
current average loss: 1.5764679908752441
current average loss: 1.5855088233947754
current average loss: 1.3350059986114502
current average loss: 1.7287275791168213
current average loss: 1.9840196371078491
current average loss: 1.6841431856155396
current average loss: 1.7081522941589355
current average loss: 1.7446821928024292
current average loss: 1.8174846172332764
current average loss: 1.2965127229690552
current average loss: 1.3029636144638062
current average loss: 1.7433843612670898
current average loss: 1.4050168991088867
current average loss: 1.411245584487915
current average loss: 1.5198493003845215
current average loss: 1.3360955715179443
current average loss: 1.7100292444229126
current average loss: 1.461759090423584
current average loss: 1.6520264148712158
current average loss: 1.493542194366455
current average loss: 1.5393562316894531
current average loss: 1.7983218431472778
current average loss: 1.6880263090133667
current average loss

...

current average loss: 2.2315785884857178
current average loss: 1.5503042936325073
current average loss: 1.5347530841827393
current average loss: 1.3465694189071655
current average loss: 1.3237510919570923
current average loss: 1.705795407295227
current average loss: 1.8137054443359375
current average loss: 1.636786699295044
current average loss: 1.5860742330551147
current average loss: 1.5910605192184448
current average loss: 1.5753990411758423
current average loss: 2.0189478397369385
current average loss: 1.9681774377822876
current average loss: 1.3852185010910034
current average loss: 1.7849098443984985
current average loss: 1.5584700107574463
current average loss: 1.3296172618865967
current average loss: 1.651464819908142
current average loss: 1.572007656097412
current average loss: 1.4227485656738281
current average loss: 1.4848934412002563
current average loss: 1.3557124137878418
current average loss: 1.517018437385559
current average loss: 1.4574702978134155
current average loss:

current average loss: 1.4039101600646973
current average loss: 1.433369755744934
current average loss: 1.7308191061019897
current average loss: 1.4417409896850586
current average loss: 1.546154499053955
current average loss: 1.8092013597488403
current average loss: 1.4263808727264404
current average loss: 1.6192876100540161
current average loss: 1.3654109239578247
current average loss: 1.8882626295089722
current average loss: 1.423926830291748
current average loss: 1.6838146448135376
current average loss: 1.3703593015670776
current average loss: 1.3977530002593994
current average loss: 1.186560034751892
current average loss: 1.2323367595672607
current average loss: 1.7384834289550781
current average loss: 1.7153247594833374
current average loss: 1.457861065864563
current average loss: 1.6099727153778076
current average loss: 1.557745099067688
current average loss: 1.648030400276184
current average loss: 1.330452561378479
current average loss: 1.4605491161346436
current average loss: 1.

current average loss: 1.6577465534210205
current average loss: 1.6128950119018555
current average loss: 1.7892694473266602
current average loss: 1.8373396396636963
current average loss: 1.4831074476242065
current average loss: 1.2972042560577393
current average loss: 1.7181419134140015
current average loss: 1.4238303899765015
current average loss: 1.5400232076644897
current average loss: 1.4618688821792603
current average loss: 1.426487922668457
current average loss: 1.630141258239746
current average loss: 1.663544774055481
current average loss: 1.2821884155273438
current average loss: 1.3359007835388184
current average loss: 1.660671591758728
current average loss: 1.6770691871643066
current average loss: 1.4423251152038574
current average loss: 1.8280839920043945
current average loss: 1.4272041320800781
current average loss: 1.3342019319534302
current average loss: 1.4933971166610718
current average loss: 1.8032288551330566
current average loss: 1.4296479225158691
current average loss

current average loss: 1.1060792207717896
current average loss: 1.655060887336731
current average loss: 1.754126787185669
current average loss: 1.5721447467803955
current average loss: 1.5930498838424683
current average loss: 1.9928969144821167
current average loss: 1.5003232955932617
current average loss: 1.8344529867172241
current average loss: 1.6307616233825684
current average loss: 1.6440680027008057
current average loss: 1.3931949138641357
current average loss: 1.479803204536438
current average loss: 1.4715489149093628
current average loss: 1.9131979942321777
current average loss: 1.6709580421447754
current average loss: 1.6971831321716309
current average loss: 1.2434614896774292
current average loss: 1.5472337007522583
current average loss: 1.5644885301589966
current average loss: 1.6370142698287964
current average loss: 1.5776970386505127
current average loss: 1.7713526487350464
current average loss: 1.409912109375
current average loss: 1.644149661064148
current average loss: 1.

current average loss: 1.6938813924789429
current average loss: 1.4123475551605225
current average loss: 1.4012757539749146
current average loss: 1.3940250873565674
current average loss: 1.5994633436203003
current average loss: 1.6325494050979614
current average loss: 1.4548648595809937
current average loss: 1.2079936265945435
current average loss: 1.8448649644851685
current average loss: 1.7984445095062256
current average loss: 1.455405592918396
current average loss: 1.5885564088821411
current average loss: 1.2355523109436035
current average loss: 1.6130170822143555
current average loss: 1.7817004919052124
current average loss: 1.3138126134872437
current average loss: 1.7243298292160034
current average loss: 1.5952606201171875
current average loss: 1.3062976598739624
current average loss: 1.764833927154541
current average loss: 1.4637441635131836
current average loss: 1.50705885887146
current average loss: 1.8974533081054688
current average loss: 1.5938800573349
current average loss: 1

In [228]:
for epoch_num in range(2000):
    params = sgd_minibatch(params, 1_000, 0.02)
    loss = avg_training_loss(params)
    print(f'current average loss: {loss}')

current average loss: 1.2542554140090942
current average loss: 1.6615387201309204
current average loss: 1.5488765239715576
current average loss: 1.4195356369018555
current average loss: 1.5964568853378296
current average loss: 1.5291332006454468
current average loss: 1.2972846031188965
current average loss: 1.557005763053894
current average loss: 1.6063416004180908
current average loss: 1.4646856784820557
current average loss: 1.3015143871307373
current average loss: 1.174456000328064
current average loss: 1.8422062397003174
current average loss: 1.5316981077194214
current average loss: 1.4480847120285034
current average loss: 1.672956943511963
current average loss: 1.7112354040145874
current average loss: 1.3611044883728027
current average loss: 1.2750403881072998
current average loss: 1.486793875694275
current average loss: 1.2459027767181396
current average loss: 1.4552509784698486
current average loss: 1.281741738319397
current average loss: 1.4435515403747559
current average loss:

current average loss: 1.4077177047729492
current average loss: 1.235691785812378
current average loss: 1.6422202587127686
current average loss: 1.1346989870071411
current average loss: 1.3484312295913696
current average loss: 1.5647534132003784
current average loss: 1.374726414680481
current average loss: 1.3596667051315308
current average loss: 1.5348129272460938
current average loss: 1.470694899559021
current average loss: 1.2007815837860107
current average loss: 1.4624016284942627
current average loss: 1.7057827711105347
current average loss: 1.5898246765136719
current average loss: 1.5734378099441528
current average loss: 1.9327083826065063
current average loss: 1.8650481700897217
current average loss: 1.5076278448104858
current average loss: 1.913985013961792
current average loss: 1.4678834676742554
current average loss: 1.376793622970581
current average loss: 2.0854320526123047
current average loss: 1.0140901803970337
current average loss: 1.1656421422958374
current average loss:

current average loss: 1.7530632019042969
current average loss: 1.557041883468628
current average loss: 1.2111872434616089
current average loss: 1.365290641784668
current average loss: 1.0799063444137573
current average loss: 1.7108352184295654
current average loss: 1.7051594257354736
current average loss: 1.5672438144683838
current average loss: 1.3512146472930908
current average loss: 1.424190878868103
current average loss: 1.653070330619812
current average loss: 1.4400341510772705
current average loss: 1.137853741645813
current average loss: 1.7781013250350952
current average loss: 1.496659278869629
current average loss: 1.8764986991882324
current average loss: 1.2403590679168701
current average loss: 1.4148772954940796
current average loss: 1.713605523109436
current average loss: 1.7044999599456787
current average loss: 1.405017375946045
current average loss: 1.4452563524246216
current average loss: 1.8055578470230103
current average loss: 1.505096435546875
current average loss: 1.1

current average loss: 1.7355786561965942
current average loss: 1.200252652168274
current average loss: 1.3241828680038452
current average loss: 1.700835108757019
current average loss: 1.5169241428375244
current average loss: 1.4399933815002441
current average loss: 1.5982916355133057
current average loss: 1.5840305089950562
current average loss: 1.5929324626922607
current average loss: 1.728043794631958
current average loss: 1.238288164138794
current average loss: 1.4725639820098877
current average loss: 1.4222228527069092
current average loss: 1.9397352933883667
current average loss: 1.6916297674179077
current average loss: 1.6650207042694092
current average loss: 1.472800612449646
current average loss: 1.247686743736267
current average loss: 1.8745253086090088
current average loss: 1.615785837173462
current average loss: 1.60310959815979
current average loss: 1.3358180522918701
current average loss: 1.4787734746932983
current average loss: 1.4021223783493042
current average loss: 1.3

current average loss: 1.5568792819976807
current average loss: 1.5240172147750854
current average loss: 1.5171666145324707
current average loss: 1.671257495880127
current average loss: 1.586999773979187
current average loss: 1.5931416749954224
current average loss: 1.4395332336425781
current average loss: 1.5792278051376343
current average loss: 1.499436378479004
current average loss: 1.599654197692871
current average loss: 1.5252983570098877
current average loss: 1.7249860763549805
current average loss: 1.4323190450668335
current average loss: 1.410436749458313
current average loss: 1.3240116834640503
current average loss: 1.4146723747253418
current average loss: 1.6892286539077759
current average loss: 1.6447564363479614
current average loss: 1.3399150371551514
current average loss: 1.795671820640564
current average loss: 1.5921030044555664
current average loss: 1.652366280555725
current average loss: 1.6198341846466064
current average loss: 1.7741470336914062
current average loss: 1

current average loss: 1.5041972398757935
current average loss: 1.5935111045837402
current average loss: 1.6384103298187256
current average loss: 1.7377668619155884
current average loss: 1.5839651823043823
current average loss: 1.2431186437606812
current average loss: 1.5057783126831055
current average loss: 1.5753226280212402
current average loss: 1.4105784893035889
current average loss: 1.3709900379180908
current average loss: 1.503567099571228
current average loss: 1.5869930982589722
current average loss: 1.7381004095077515
current average loss: 1.5178571939468384
current average loss: 1.2425916194915771
current average loss: 1.4737913608551025
current average loss: 1.1552767753601074
current average loss: 1.7554222345352173
current average loss: 1.555496096611023
current average loss: 1.1016181707382202
current average loss: 1.4961624145507812
current average loss: 1.6774100065231323
current average loss: 1.5830438137054443
current average loss: 1.37786865234375
current average loss

current average loss: 1.590948224067688
current average loss: 1.4747220277786255
current average loss: 1.2927204370498657
current average loss: 1.7564586400985718
current average loss: 1.8345106840133667
current average loss: 1.6006032228469849
current average loss: 1.3231121301651
current average loss: 1.5936886072158813
current average loss: 1.0113606452941895
current average loss: 1.390579104423523
current average loss: 1.7316491603851318
current average loss: 1.6108664274215698
current average loss: 1.4533882141113281
current average loss: 1.4294390678405762
current average loss: 1.4860103130340576
current average loss: 1.5209087133407593
current average loss: 1.3358503580093384
current average loss: 1.128158688545227
current average loss: 1.340035080909729
current average loss: 1.2518242597579956
current average loss: 1.6354832649230957
current average loss: 1.324831247329712
current average loss: 1.2954891920089722
current average loss: 0.9991389513015747
current average loss: 1.

current average loss: 1.3188214302062988
current average loss: 1.1582592725753784
current average loss: 1.6282387971878052
current average loss: 1.473167061805725
current average loss: 1.3230150938034058
current average loss: 1.2424097061157227
current average loss: 1.4905592203140259
current average loss: 1.4058743715286255
current average loss: 1.5992119312286377
current average loss: 1.451075553894043
current average loss: 1.79823899269104
current average loss: 1.7857416868209839
current average loss: 1.6502004861831665
current average loss: 1.5477365255355835
current average loss: 1.79835844039917
current average loss: 1.6673707962036133
current average loss: 1.6131385564804077
current average loss: 1.4014570713043213
current average loss: 1.6702574491500854
current average loss: 1.4981379508972168
current average loss: 1.2999409437179565
current average loss: 1.7325245141983032
current average loss: 1.6725587844848633
current average loss: 1.8609853982925415
current average loss: 

current average loss: 1.400646686553955
current average loss: 1.199431300163269
current average loss: 1.4606019258499146
current average loss: 1.3187295198440552
current average loss: 1.4824118614196777
current average loss: 1.3342214822769165
current average loss: 1.4434380531311035
current average loss: 1.242753267288208
current average loss: 1.8629660606384277
current average loss: 1.5912392139434814
current average loss: 1.3374395370483398
current average loss: 1.6248257160186768
current average loss: 1.5247867107391357
current average loss: 1.4594889879226685
current average loss: 1.8265380859375
current average loss: 1.8351211547851562
current average loss: 1.6306685209274292
current average loss: 1.4024139642715454
current average loss: 1.7802445888519287
current average loss: 1.4654343128204346
current average loss: 1.6039777994155884
current average loss: 1.339876413345337
current average loss: 1.487351417541504
current average loss: 1.9118685722351074
current average loss: 1.

current average loss: 1.5161161422729492
current average loss: 1.4361625909805298
current average loss: 1.5995674133300781
current average loss: 1.5284053087234497
current average loss: 1.5100462436676025
current average loss: 1.5516149997711182
current average loss: 1.8106173276901245
current average loss: 1.5391660928726196
current average loss: 1.1196914911270142
current average loss: 1.4904171228408813
current average loss: 1.7609504461288452
current average loss: 1.3392966985702515
current average loss: 1.4164249897003174
current average loss: 1.5143914222717285
current average loss: 1.1997261047363281
current average loss: 1.4891579151153564
current average loss: 1.7497984170913696
current average loss: 1.3197728395462036
current average loss: 1.6349372863769531
current average loss: 1.2964608669281006
current average loss: 1.7351112365722656
current average loss: 1.370253086090088
current average loss: 1.4037225246429443
current average loss: 1.465022087097168
current average lo

In [229]:
for epoch_num in range(2000):
    params = sgd_minibatch(params, 1_000, 0.001)
    loss = avg_training_loss(params)
    print(f'current average loss: {loss}')

current average loss: 1.4608263969421387
current average loss: 1.494523286819458
current average loss: 1.478373408317566
current average loss: 1.312852382659912
current average loss: 1.4277396202087402
current average loss: 1.250531792640686
current average loss: 1.513982892036438
current average loss: 1.3503199815750122
current average loss: 1.4955034255981445
current average loss: 1.5765827894210815
current average loss: 1.5134599208831787
current average loss: 1.4589753150939941
current average loss: 1.669905185699463
current average loss: 1.3594176769256592
current average loss: 1.234800934791565
current average loss: 1.6163618564605713
current average loss: 1.6656384468078613
current average loss: 1.4963431358337402
current average loss: 1.3426504135131836
current average loss: 1.4957906007766724
current average loss: 1.2272326946258545
current average loss: 1.4340168237686157
current average loss: 1.4909920692443848
current average loss: 1.2259156703948975
current average loss: 1

current average loss: 1.2529348134994507
current average loss: 1.16493558883667
current average loss: 1.3618513345718384
current average loss: 1.5846551656723022
current average loss: 1.4469854831695557
current average loss: 1.317307472229004
current average loss: 1.4798065423965454
current average loss: 1.2179419994354248
current average loss: 1.4984363317489624
current average loss: 1.4625695943832397
current average loss: 1.2577438354492188
current average loss: 1.6702266931533813
current average loss: 1.5225046873092651
current average loss: 1.3624470233917236
current average loss: 1.4374717473983765
current average loss: 1.6649174690246582
current average loss: 1.5527290105819702
current average loss: 1.5519616603851318
current average loss: 1.2711007595062256
current average loss: 1.6506712436676025
current average loss: 1.4323420524597168
current average loss: 1.4131561517715454
current average loss: 1.4397183656692505
current average loss: 1.2905181646347046
current average los

current average loss: 1.6277018785476685
current average loss: 1.3843541145324707
current average loss: 1.2616219520568848
current average loss: 1.6545318365097046
current average loss: 1.33943510055542
current average loss: 1.2856478691101074
current average loss: 1.8549619913101196
current average loss: 1.5753161907196045
current average loss: 1.5571584701538086
current average loss: 1.7391265630722046
current average loss: 1.6768258810043335
current average loss: 1.4647643566131592
current average loss: 1.4827781915664673
current average loss: 1.6285547018051147
current average loss: 1.8950523138046265
current average loss: 1.4201648235321045
current average loss: 1.2247741222381592
current average loss: 1.3326809406280518
current average loss: 1.9511587619781494
current average loss: 1.2413990497589111
current average loss: 1.2812443971633911
current average loss: 1.558620572090149
current average loss: 1.341418743133545
current average loss: 1.5425699949264526
current average loss

current average loss: 1.1240495443344116
current average loss: 1.3389936685562134
current average loss: 1.400489091873169
current average loss: 1.636875033378601
current average loss: 1.3450307846069336
current average loss: 1.6510605812072754
current average loss: 1.2760839462280273
current average loss: 1.4045376777648926
current average loss: 1.5077422857284546
current average loss: 1.5810469388961792
current average loss: 1.6021511554718018
current average loss: 1.441477656364441
current average loss: 1.505515694618225
current average loss: 1.5473756790161133
current average loss: 1.4389195442199707
current average loss: 1.352911114692688
current average loss: 1.4960289001464844
current average loss: 1.5321179628372192
current average loss: 1.5159469842910767
current average loss: 1.5812376737594604
current average loss: 1.396414041519165
current average loss: 1.4179701805114746
current average loss: 1.7370381355285645
current average loss: 1.5661184787750244
current average loss: 

current average loss: 1.1797751188278198
current average loss: 1.5078761577606201
current average loss: 1.7057502269744873
current average loss: 1.348004937171936
current average loss: 1.3075488805770874
current average loss: 1.2687424421310425
current average loss: 1.5712405443191528
current average loss: 1.7113374471664429
current average loss: 1.4284653663635254
current average loss: 1.4523093700408936
current average loss: 1.7018970251083374
current average loss: 1.4933836460113525
current average loss: 1.781153917312622
current average loss: 1.3911943435668945
current average loss: 1.5759460926055908
current average loss: 1.2348817586898804
current average loss: 1.2517421245574951
current average loss: 1.2195806503295898
current average loss: 1.5354727506637573
current average loss: 1.4596563577651978
current average loss: 1.3253111839294434
current average loss: 1.4241557121276855
current average loss: 1.2801088094711304
current average loss: 1.60732102394104
current average loss

current average loss: 1.609547734260559
current average loss: 1.4944342374801636
current average loss: 1.3648693561553955
current average loss: 1.3118982315063477
current average loss: 1.5125982761383057
current average loss: 1.7486793994903564
current average loss: 1.278520941734314
current average loss: 1.6984336376190186
current average loss: 1.689324140548706
current average loss: 1.6938562393188477
current average loss: 1.116531252861023
current average loss: 1.518856167793274
current average loss: 1.6383172273635864
current average loss: 1.421647548675537
current average loss: 1.76802659034729
current average loss: 1.814040184020996
current average loss: 1.6402788162231445
current average loss: 1.3573330640792847
current average loss: 1.2341216802597046
current average loss: 1.5155972242355347
current average loss: 1.494676947593689
current average loss: 1.4831658601760864
current average loss: 1.1733919382095337
current average loss: 1.613050937652588
current average loss: 1.418

current average loss: 1.4908593893051147
current average loss: 1.4949088096618652
current average loss: 1.432948350906372
current average loss: 1.549574375152588
current average loss: 1.748802900314331
current average loss: 1.3066258430480957
current average loss: 1.599794864654541
current average loss: 1.406270146369934
current average loss: 1.4788800477981567
current average loss: 1.5280120372772217
current average loss: 1.2873332500457764
current average loss: 1.4520366191864014
current average loss: 1.6967178583145142
current average loss: 1.7165746688842773
current average loss: 1.6343700885772705
current average loss: 1.5302554368972778
current average loss: 1.459842324256897
current average loss: 1.1451069116592407
current average loss: 1.6212143898010254
current average loss: 1.5937385559082031
current average loss: 1.6489235162734985
current average loss: 1.352593183517456
current average loss: 1.608061671257019
current average loss: 1.5285605192184448
current average loss: 1.

current average loss: 1.587025761604309
current average loss: 1.768931269645691
current average loss: 1.5105922222137451
current average loss: 1.178961992263794
current average loss: 1.346234917640686
current average loss: 1.548235297203064
current average loss: 1.5692644119262695
current average loss: 1.3159183263778687
current average loss: 1.2994558811187744
current average loss: 1.2760534286499023
current average loss: 1.4361538887023926
current average loss: 1.2393040657043457
current average loss: 1.5265617370605469
current average loss: 1.507216453552246
current average loss: 1.2785825729370117
current average loss: 1.610135793685913
current average loss: 1.529615879058838
current average loss: 1.4205759763717651
current average loss: 1.2944053411483765
current average loss: 1.4559807777404785
current average loss: 1.7067104578018188
current average loss: 1.824865698814392
current average loss: 1.1903661489486694
current average loss: 1.3211390972137451
current average loss: 1.5

current average loss: 1.5716899633407593
current average loss: 1.7498846054077148
current average loss: 1.5767695903778076
current average loss: 1.3386168479919434
current average loss: 1.6408565044403076
current average loss: 1.1457842588424683
current average loss: 1.4487971067428589
current average loss: 1.4882889986038208
current average loss: 1.3951921463012695
current average loss: 1.4205543994903564
current average loss: 1.6514790058135986
current average loss: 1.6056158542633057
current average loss: 1.4975634813308716
current average loss: 1.639755368232727
current average loss: 1.5519778728485107
current average loss: 1.5297030210494995
current average loss: 1.6302075386047363
current average loss: 1.624544382095337
current average loss: 1.8268235921859741
current average loss: 1.6354464292526245
current average loss: 1.5924851894378662
current average loss: 1.3562849760055542
current average loss: 1.3877445459365845
current average loss: 1.3930920362472534
current average lo

current average loss: 1.388924241065979
current average loss: 1.4788647890090942
current average loss: 1.2617955207824707
current average loss: 1.6661367416381836
current average loss: 1.5194662809371948
current average loss: 1.4759997129440308
current average loss: 1.5759332180023193
current average loss: 1.643081784248352
current average loss: 1.451161503791809
current average loss: 1.676061749458313
current average loss: 1.5570217370986938
current average loss: 1.4302799701690674
current average loss: 1.5791444778442383
current average loss: 1.6431657075881958
current average loss: 1.235146164894104
current average loss: 1.4261937141418457
current average loss: 1.5855647325515747
current average loss: 1.1595141887664795
current average loss: 1.5572694540023804
current average loss: 1.5551633834838867
current average loss: 1.2655205726623535
current average loss: 2.0680806636810303
current average loss: 1.449368953704834
current average loss: 1.8219871520996094
current average loss: 